# Cargar maestro_picks_v2 en Postgres

Este notebook carga `df_maestro/maestro_picks_v2.pkl` a una tabla Postgres usando SQLAlchemy.
Asegura `DATABASE_URL` en tu `.env` o define la variable en el notebook.


In [1]:
import os
import sys
from pathlib import Path

import pandas as pd

REPO_PATH = Path("/home/pablo/Documentos/footystats")
if REPO_PATH.exists():
    sys.path.insert(0, str(REPO_PATH))
    os.chdir(REPO_PATH)

def resolve_maestro() -> Path:
    cand = REPO_PATH / "df_maestro" / "maestro_picks_v2.pkl"
    if cand.exists():
        return cand
    raise FileNotFoundError(f"No se encontró maestro_picks_v2.pkl en {cand}")

MAESTRO_PATH = resolve_maestro()
df = pd.read_pickle(MAESTRO_PATH)
df.shape, MAESTRO_PATH


((403, 48),
 PosixPath('/home/pablo/Documentos/footystats/df_maestro/maestro_picks_v2.pkl'))

In [2]:
import os
from pathlib import Path
from sqlalchemy import create_engine, text

def _find_env(start: Path) -> Path | None:
    for p in [start] + list(start.parents):
        cand = p / ".env"
        if cand.exists():
            return cand
    return None

def _read_env_var(key: str) -> str:
    if key in os.environ and os.environ[key]:
        return os.environ[key]
    repo = Path("/home/pablo/Documentos/footystats")
    env_path = repo / ".env" if repo.exists() else None
    if env_path is None or not env_path.exists():
        env_path = _find_env(Path.cwd())
    if env_path and env_path.exists():
        for raw in env_path.read_text(encoding="utf-8").splitlines():
            line = raw.strip()
            if not line or line.startswith("#") or "=" not in line:
                continue
            k, v = line.split("=", 1)
            if k.strip() == key:
                return v.strip().strip(""").strip("'")
    return ""

DATABASE_URL = _read_env_var("DATABASE_URL")
if not DATABASE_URL:
    raise RuntimeError("DATABASE_URL no está definido. Agrega la variable en .env o en el entorno.")

engine = create_engine(DATABASE_URL, pool_pre_ping=True)
with engine.connect() as conn:
    conn.execute(text("select 1"))
"Conexion OK"


RuntimeError: DATABASE_URL no está definido. Agrega la variable en .env o en el entorno.

In [ ]:
table_name = "maestro_picks_v2"
if_exists = "append"  # usa "replace" en la primera carga si quieres recrear la tabla

df.to_sql(
    table_name,
    engine,
    index=False,
    if_exists=if_exists,
    method="multi",
    chunksize=1000,
)
f"Cargadas {len(df)} filas en {table_name}"


In [ ]:
with engine.connect() as conn:
    total = conn.execute(text(f"select count(*) from {table_name}")).scalar()
total
